## Try out Microsoft Phi model (Inferencing and Fine tuning)

In [53]:
# Install required packages

%pip install einops
%pip install datasets
%pip install scikit-learn
%pip install accelerate -U


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


### Load the model using Transformer Pipeline

In [104]:
# Use a pipeline as a high-level helper
from transformers import pipeline
model_name = "gpt2"
#model_name = "microsoft/phi-2"

pipe = pipeline("text-generation", model=model_name, trust_remote_code=True)

/Users/omkarmore/anaconda3/envs/transformers/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [105]:
txt = pipe('Here is a short childrens story. Once upon a time there was a thristy crow', max_length=512)[0]['generated_text']

print(txt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Here is a short childrens story. Once upon a time there was a thristy crow. The thristy crow was a tiny little animal with a human face, but no eyes or ears. Some people described it as being like a chicken, but it never came off the chicken. It was so tiny it had to burrow under the skin of a young child and live in the earth. The kid couldn't chew food and that was good news because it was such a small animal.

"In the beginning of the book he describes how his little chicken was so small, so small, that if he got too much he could crawl, but there was no way he could take his food and eat it, because there was no water inside, and there was no food. And when he died, the little chicken said, 'My life is over.' So my little bird died and didn't remember he was there. It was one of the only creatures that never grew old. There was always a problem with that one. When there is one thing you need to feed in large quantities it is always one that feeds in small amounts. And it was this t

## Finetune a Phi model for Text Generation Task

In [106]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load a dataset (replace with your dataset)
dataset = load_dataset("euclaise/writingprompts")


In [107]:
import pandas as pd

sampled_dataset = dataset['train'].select(range(5))
df = pd.DataFrame(sampled_dataset)
print(df)

                                              prompt  \
0  [ WP ] You 've finally managed to discover the...   
1  [ WP ] The moon is actually a giant egg , and ...   
2  [ WP ] You find a rip in time walking through ...   
3  [ WP ] For years in your youth the same imagin...   
4  [ WP ] You glance at your watch 10:34 am , rou...   

                                               story  
0  So many times have I walked on ruins, the rema...  
1  -Week 18 aboard the Depth Reaver, Circa 2023- ...  
2  I was feckin' sloshed, mate. First time I ever...  
3  “ No, no no no... ” She backed up and turned t...  
4  There's a magical moment between wakefulness a...  


In [108]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Concatenate the prompt and story
    concatenated_texts = [prompt + " " + story for prompt, story in zip(examples['prompt'], examples['story'])]
    tokenized_inputs = tokenizer(concatenated_texts, padding='max_length', truncation=True, max_length=512)  # Adjust max_length as needed
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 15138/15138 [00:07<00:00, 2029.47 examples/s]


In [109]:
# Sample 10,000 entries
sampled_dataset_dict = {split: tokenized_datasets[split].shuffle(seed=42).select(range(100))
                        for split in tokenized_datasets.keys()}

In [110]:
sampled_dataset = sampled_dataset_dict['train'].select(range(5))
df = pd.DataFrame(sampled_dataset)
print(df)


                                              prompt  \
0  Light bulbs have gotten efficient enouh to las...   
1  [ WP ] Just write about something that makes y...   
2  [ WP ] With the discovery of alternate dimensi...   
3  [ WP ] A man has been waiting for his daughter...   
4  [ EU ] Before Bruce Wayne 's death , he create...   

                                               story  \
0  The three men stood in silence and stared up a...   
1  I ’ ve learned to appreciate all kinds of beau...   
2  Edit: I wrote this in mind for Blake Andersen ...   
3  Riley sits in his padded cell with his back to...   
4  It started when I was 8. I had been staying ov...   

                                           input_ids  \
0  [15047, 34122, 423, 7891, 6942, 551, 280, 71, ...   
1  [58, 28993, 2361, 2329, 3551, 546, 1223, 326, ...   
2  [58, 28993, 2361, 2080, 262, 9412, 286, 13527,...   
3  [58, 28993, 2361, 317, 582, 468, 587, 4953, 32...   
4  [58, 4576, 2361, 7413, 11088, 13329, 705, 8

In [111]:
# Load model directly
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

In [112]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./{model_name}-finetuned",
    per_device_train_batch_size=1,  # Adjust batch size according to your GPU memory
   # gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
)



In [113]:
sampled_dataset_dict

{'train': Dataset({
     features: ['prompt', 'story', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }),
 'test': Dataset({
     features: ['prompt', 'story', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }),
 'validation': Dataset({
     features: ['prompt', 'story', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 })}

In [114]:
from transformers import Trainer

tokenized_datasets

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sampled_dataset_dict["train"],  # Replace with your train dataset
    eval_dataset=sampled_dataset_dict["validation"]  # Replace with your validation dataset, if available
)


In [115]:
trainer.train()


  0%|          | 0/300 [00:00<?, ?it/s]

  4%|▎         | 11/300 [00:05<00:57,  4.99it/s]

{'loss': 4.0507, 'learning_rate': 4.8333333333333334e-05, 'epoch': 0.1}


  7%|▋         | 21/300 [00:06<00:47,  5.89it/s]

{'loss': 3.5772, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.2}


...

 97%|█████████▋| 291/300 [00:52<00:01,  5.91it/s]

{'loss': 2.7133, 'learning_rate': 1.6666666666666667e-06, 'epoch': 2.9}


100%|██████████| 300/300 [00:54<00:00,  5.52it/s]

{'loss': 2.681, 'learning_rate': 0.0, 'epoch': 3.0}
{'train_runtime': 54.3664, 'train_samples_per_second': 5.518, 'train_steps_per_second': 5.518, 'train_loss': 2.8851855278015135, 'epoch': 3.0}


TrainOutput(global_step=300, training_loss=2.8851855278015135, metrics={'train_runtime': 54.3664, 'train_samples_per_second': 5.518, 'train_steps_per_second': 5.518, 'train_loss': 2.8851855278015135, 'epoch': 3.0})

In [117]:
model.save_pretrained(f"./{model_name}-finetuned/")
tokenizer.save_pretrained(f"./{model_name}-finetuned/")

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json',
 './gpt2-finetuned/tokenizer.json')

In [118]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer and model
model_name = f"./{model_name}-finetuned"  # Replace with your model directory
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [120]:
# Use a pipeline as a high-level helper
from transformers import pipeline

model_name = f"./gpt2-finetuned"

pipe = pipeline("text-generation", model=model_name, trust_remote_code=True)

In [122]:
txt = pipe('Here is a short childrens story. Once upon a time there was a thristy crow', max_length=512)[0]['generated_text']

print(txt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Here is a short childrens story. Once upon a time there was a thristy crow's nest all over the place, and all of it lay dark and silent, the sun setting a crisp green over the bright woods. The thristy crows never fell asleep. The only darkness was the trees. They grew into trees, and the thirst had given way to the light. Brightly lit, light filled the darkness. The last of the trees grew dimly, and then dark and lifeless. The trees burned and the sun lit the darkness even more. They bore faint scars and scars that they could not make out and did it all over again. Sometimes I heard whispers of the tree that burned to the ground. It was the last of the trees, and the trees that grew old were forgotten and nothing truly remained of them. The tree fell into the forest just like the last time, and then the lights of the dark had returned.

